In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from mmcv.image import imread
from mmpose.apis import inference_topdown, init_model
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples
from tqdm import tqdm
import os, json, glob, shutil, cv2
import copy
import numpy as np
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

#labeling
center = ["Nostril",
            "Eye",
            "Poll",
            'Withers',
            'LowestBack',
            'T16L1'
             ,'T_sacrale'
             ,'Tail_Root'
             ,'T_ischiadicum'
             ,'Tub'
             ,'Spina_scapulae'
             ,'ElbowJoint_L'
             ,'ElbowJoint_R'
             ,'Carpuse_L'
             ,'Carpuse_R'
             ,'Fetlock_L'
             ,'Fetlock_R'
             ,'Front_Heel_L'
             ,'Front_Heel_R'
             ,'Front_Toe_L'
             ,'Front_Toe_R'
             ,'Abdomen'
             ,'T_Coxae'
             ,'Coxofemoral'
             ,'Stifle_Joint_L'
             ,'Stifle_Joint_R'
             ,'Rear_Tarsus_L'
             ,'Rear_Tarsus_R'
             ,'Rear_Fetlock_L'
             ,'Rear_Fetlock_R'
             ,'Rear_Heel_L'
             ,'Rear_Heel_R'
             ,'Rear_Toe_L'
             ,"Rear_Toe_R"]

# YOLO 모델 로드
yolo_model = YOLO('yolov10m.pt')  # custom한 YOLOv8 모델 사용

#rtmpose 모델 로드
pose_config = 'Test/rtmpose-m_8xb256-420e_coco-256x192.py'
pose_checkpoint = 'Test/epoch_550.pth'
device = 'cpu'
rtm_model = init_model(pose_config, pose_checkpoint, device=device)


# 디렉토리 설정
folder = '5_2_del'
image_dir = f'/RnD/horse/{folder}/'  # 이미지
output_dir = f'/RnD/horse/{folder}'  # rtmpose 결과 저장

# 출력 디렉토리가 존재하지 않으면 생성
os.makedirs(output_dir, exist_ok=True)

# 지원할 이미지 파일 확장자
image_extensions = ['*.jpg', '*.jpeg', '*.png']

# 모든 이미지 파일 경로 가져오기
image_paths = []
for ext in image_extensions:
    image_paths.extend(glob.glob(os.path.join(image_dir, ext)))

# df 셍상
hdf = pd.read_hdf('CollectedData_teamDLC.h5')
index = pd.MultiIndex.from_product([
    ['horse'],
    [folder],
    [img_path.split('/')[-1] for img_path in image_paths]
])

dfs = pd.DataFrame(index = index, columns = hdf.columns)

# (5) 필터링 된 이미지 파일 처리 및 바운딩 박스 정보 저장
for idx, img_path in enumerate(image_paths):
    # 이미지 로드
    image = cv2.imread(img_path)
    if image is None:
        print(f"Image not found: {img_path}")
        continue
    height,width,_=image.shape #이미지 height,width 측정 
    # 객체 탐지
    results = yolo_model(image)
    #1. 바운딩 박스 정보 저장
    for result in results:
        boxes = result.boxes  # 바운딩 박스 정보
        #print(boxes)
        x1, y1 = int(0.25 * width), int(0.25 * height) # undifined방지> 바운딩 박스 기본값 설정 (이미지 중앙 부분)
        x2, y2 = int(0.75 * width), int(0.75 * height)
        boxcal_data=[[x1,y1],[x2,y2]] #bbox
        if boxes is None: 
            continue
        for box in boxes:
            if int(box.cls) == 17:  # horse class
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                boxcal_data=[[x1,y1],[x2,y2]] #bbox
                #2. 바운딩 박스 확장 (좌우 5%씩)
                box_width = x2 - x1
                box_height = y2 - y1
                x1 = int(x1 - 0.05 * width)
                y1 = int(y1 - 0.05 * height)
                x2 = int(x2 + 0.05 * width)
                y2 = int(y2 + 0.05 * height)

                # 이미지 범위를 벗어나지 않도록 조정
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(image.shape[1], x2)
                y2 = min(image.shape[0], y2)
                break  # 첫 번째 말 객체만 처리
    #print(x1, y1, x2, y2)

    #3. 이미지 크롭하기
    cropped_image = image[y1:y2, x1:x2]

    #4. 이미지 리사이즈 256X192
    rtmpose_size=(256,192)
    cropped_image=cv2.resize(cropped_image,rtmpose_size)

    #크롭한 이미지 비주얼화
    # plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
    # plt.title('Cropped Image')
    # plt.show()

    # 5. rtmpose 적용하기 -keypoint 가져오기 
    batch_results=inference_topdown(rtm_model,cropped_image)
    #print(batch_results[0].pred_instances.keypoints_visible)
    #min-max 정규화->데이터최대최소를 설정해서 그안에서 상대적인 대소관계유지 
    kpntscores=batch_results[0].pred_instances.keypoints_visible
    min_num=min(kpntscores[0])
    max_num=max(kpntscores[0])
    normalized_scores=(kpntscores[0]-min_num)/(max_num-min_num)
    #print(normalized_scores)
    # 키포인트를 크롭된 이미지에 찍기
    keypnt=batch_results[0].pred_instances.keypoints[0]
    keypnt_data=keypnt.tolist()

    #cnn 모델 ------------------------------------ 
    # 좌표 색상(픽셀값) 확인하기
    def classify_color(color):
        R, G, B = color[2], color[1], color[0]
        
        if 200 <= R <= 255 and 200 <= G <= 255 and 200 <= B <= 255:
            return 'White'
        elif 150 <= R <= 255 and 50 <= G <= 200 and 150 <= B <= 255:
            return 'Pink'
        elif 150 <= R <= 255 and 0 <= G <= 100 and 0 <= B <= 100:
            return 'Red'
        elif 80 <= R <= 200 and 40 <= G <= 150 and 0 <= B <= 100:
            return 'Brown'
        else: #그 외의 색상이나 이미지가 범위에 들어가지 않는경우 일괄로 others로 설정 
            return 'Other'
    
    #픽셀 값: 각 픽셀의 색상과 밝기 정의 (RGB에서는 각 픽셀은 세가지 색상 채널의 값을 가짐 red,green,blue)
    label_color=[]
    for i in range(34):
        x, y = int(keypnt_data[i][0]), int(keypnt_data[i][1]) #정수화->특정 좌표의 픽셀 값을 가져올 때 정수형 좌표가 필요(소수 안됨)
        if 0 <= x < cropped_image.shape[1] and 0 <= y < cropped_image.shape[0]:  # 좌표가 이미지 범위 내에 있는지 확인
            color = cropped_image[y, x]
            label_color.append(classify_color(color))
        else:
            label_color.append('Other')

    #픽셀값과 임계값에 따라서 visible 결정하기 
    label_id=[]
    for i in range(34): #라벨링 0<->1이나 1<->2는 있지만 0<->2인 경우는 없으므로 따로 만들지 않음  
        if i==3 or i==11: # u4 d4는 2로
            #1 2 3 4 5 6 7 8 1 2 3 4 5 6 7 8 9
            label_id.append(2)
        elif normalized_scores[i]>=0.6 : #2
            label_id.append(2)
        elif normalized_scores[i]>=0.3 : #1
            if label_color[i]=='Pink' or label_color[i]=='Red' or label_color[i]=='Brown': #1인테 핑크나 갈색인경우 0으로 (입술이나 잇몸)
                #print('change value 1 to 0 at',i)
                label_id.append(0) 
                continue
            label_id.append(1)
        else: #0
            if label_color[i]=='White':#0인데 흰색인 경우 (치아일 가능성이 있음)
                #print('change value 0 to 1 at',i)
                label_id.append(1)
                continue
            label_id.append(0)
                  

    # 크롭된 이미지의 키포인트를 원본 이미지의 좌표계로 변환
    scale_x = (x2 - x1) / rtmpose_size[0]
    scale_y = (y2 - y1) / rtmpose_size[1]
    keypoints_original = np.array(keypnt_data) * [scale_x, scale_y] + [x1, y1]
    # 원본 이미지에 키포인트 시각화
    for point in keypoints_original:
        cv2.circle(image, (int(point[0]), int(point[1])), 5, (0, 0, 255), -1)
    # 키포인트를 적용한 크롭된 이미지 시각화
    cv2.rectangle(image,(boxcal_data[0][0],boxcal_data[0][1]),(boxcal_data[1][0],boxcal_data[1][1]),color=(255,0,0), thickness=2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()

    # keypoints_original이 리스트가 아닌 경우 (예: numpy 배열), 리스트로 변환
    keypoints_original = keypoints_original.tolist() if isinstance(keypoints_original, np.ndarray) else keypoints_original

    for i, keypoint in enumerate(keypoints_original):
        col_index = i * 2  
        if col_index + 1 < dfs.shape[1]:
            dfs.iloc[idx , col_index] = float(keypoint[0])  # x 좌표
            dfs.iloc[idx , col_index + 1] = float(keypoint[1])  # y 좌표


dfs = dfs.astype(float)
dfs = dfs[(dfs >= 0).all(axis=1)]
x_condition = (dfs.xs('x', level=2, axis=1) <= 1920).all(axis=1)
y_condition = (dfs.xs('y', level=2, axis=1) <= 1080).all(axis=1)

# 두 조건을 모두 만족하는 행만 선택
dfs = dfs[x_condition & y_condition]
path = os.path.join(image_dir, f'CollectedData_teamDLC_550.h5')
dfs.to_hdf(path, key='df_with_missing', mode='w')

ImportError: /opt/conda/lib/python3.10/site-packages/mmcv/_ext.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZNK3c106SymIntltEl

In [4]:
# 1. 기존 설치 제거
pip uninstall mmcv-full
pip uninstall mmcv

# 2. PyTorch 버전 확인
python -c "import torch; print(torch.__version__)"

# 3. 호환되는 mmcv-full 설치
# PyTorch 1.x 버전용
pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html

# PyTorch 2.x 버전용
pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html

SyntaxError: invalid syntax (3173717473.py, line 2)

In [6]:
hdf_ai = pd.read_hdf('/RnD/horse/5_2_del/CollectedData_teamDLC_550.h5')

In [7]:
hdf_ai 

scorer                            teamDLC                           \
bodyparts                         Nostril                      Eye   
coords                                  x           y            x   
horse 5_2_del image_5469.jpg    82.033203  509.776042   134.184900   
              image_5857.jpg  1882.896503  490.937505  1839.966797   
              image_906.jpg    749.933594  514.484375   783.380863   
              image_560.jpg   1098.932966  647.402787  1073.690430   
              image_5839.jpg  1309.444316  476.003906  1259.493144   
...                                   ...         ...          ...   
              image_5441.jpg   863.216797  476.666663   918.555343   
              image_1331.jpg  1784.886719  499.906255  1738.304688   
              image_909.jpg    676.101562  524.604163   708.289062   
              image_608.jpg   1893.404969  625.388898  1882.906922   
              image_893.jpg   1024.885421  513.113281  1050.892899   

scorer                                                             \
bodyparts                                        Poll               
coords                                 y            x           y   
horse 5_2_del image_5469.jpg  454.209635   176.470056  420.869792   
              image_5857.jpg  412.812495  1793.970703  365.937502   
              image_906.jpg   466.177083   815.490238  424.770833   
              image_560.jpg   554.833333  1029.030623  493.618053   
              image_5839.jpg  409.944873  1214.537089  357.097656   
...                                  ...          ...         ...   
              image_5441.jpg  437.708337   958.675781  407.916668   
              image_1331.jpg  412.953120  1686.546875  360.453125   
              image_909.jpg   474.187500   747.182288  435.229168   
              image_608.jpg   530.315104  1839.165039  470.345484   
              image_893.jpg   464.167969  1082.375655  423.380207   

scorer                                                              \
bodyparts                         Withers               LowestBack   
coords                                  x           y            x   
horse 5_2_del image_5469.jpg   363.934253  490.019093   424.542969   
              image_5857.jpg  1544.058575  447.187500  1475.064434   
              image_906.jpg    989.416024  482.279510  1048.283195   
              image_560.jpg    683.402344  462.263887   603.791329   
              image_5839.jpg   999.747050  461.140625   926.485352   
...                                   ...         ...          ...   
              image_5441.jpg  1113.623689  478.958337  1168.962248   
              image_1331.jpg  1431.208312  455.609380  1358.747406   
              image_909.jpg    937.625000  489.083337   989.929688   
              image_608.jpg   1504.977235  458.644099  1420.992860   
              image_893.jpg   1265.796875  474.656250  1321.918303   

scorer                                    ...                             \
bodyparts                                 ... Rear_Fetlock_R               
coords                                 y  ...              x           y   
horse 5_2_del image_5469.jpg  507.306427  ...     655.701806  785.138470   
              image_5857.jpg  469.062495  ...    1177.623047  873.750000   
              image_906.jpg   499.532115  ...    1251.642578  775.573771   
              image_560.jpg   484.659720  ...     388.259103  947.506926   
              image_5839.jpg  484.261280  ...     680.059580  895.478712   
...                                  ...  ...            ...         ...   
              image_5441.jpg  497.291663  ...    1296.240902  750.520847   
              image_1331.jpg  475.296880  ...    1065.453125  893.656270   
              image_909.jpg   503.979163  ...    1124.044287  766.375000   
              image_608.jpg   480.584204  ...    1219.780273  941.326371   
              image_893.jpg   486.309899  ...    1417.735352  779.981785   

scorer

In [345]:
hdf_me = pd.read_hdf('/RnD/horse/2_1_2_del/CollectedData_teamDLC.h5')

In [346]:
import pandas as pd
import re

# df는 여러분의 DataFrame이라고 가정합니다

def extract_number(filename):
    # 파일명에서 숫자를 추출합니다
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else 0

# 세 번째 레벨(파일명)을 기준으로 정렬합니다
hdf_ai = hdf_ai.sort_index(level=2, key=lambda x: x.map(extract_number))
hdf_me = hdf_me.sort_index(level=2, key=lambda x: x.map(extract_number))

In [355]:
pd.concat([hdf_me,hdf_ai[180:]]).to_hdf('/RnD/horse/2_1_2_del/CollectedData_teamDLC_sum.h5', key='df_with_missing', mode='w')

In [373]:
hdf_sum = pd.read_hdf('/RnD/horse/2_1_2_del/CollectedData_teamDLC_sum.h5');hdf_sum

scorer                              teamDLC                           \
bodyparts                           Nostril                      Eye   
coords                                    x           y            x   
horse 2_1_2_del image_229.jpg    690.531081  501.885712   636.097797   
                image_230.jpg    693.976225  504.641828   642.988086   
                image_231.jpg    703.622630  494.306394   651.945462   
                image_232.jpg    707.067775  495.684452   654.012548   
                image_233.jpg    715.336122  491.550279   660.213809   
...                                     ...         ...          ...   
                image_1259.jpg  1754.793945  484.874562  1711.190444   
                image_1260.jpg  1759.372382  484.611107  1714.938788   
                image_1261.jpg  1763.226549  487.203125  1722.093736   
                image_1262.jpg  1768.726562  486.710942  1724.991211   
                image_1263.jpg  1771.359375  486.175351  1728.322266   

scorer                                                               \
bodyparts                                          Poll               
coords                                   y            x           y   
horse 2_1_2_del image_229.jpg   439.873110   600.957322  403.354578   
                image_230.jpg   435.049908   630.585565  398.531375   
                image_231.jpg   434.360879   629.896536  397.842347   
                image_232.jpg   435.049908   630.585565  401.287491   
                image_233.jpg   435.738937   634.719739  396.464289   
...                                    ...          ...         ...   
                image_1259.jpg  427.841146  1681.356431  393.064672   
                image_1260.jpg  427.666667  1686.455743  394.333333   
                image_1261.jpg  429.143229  1692.386719  395.966146   
                image_1262.jpg  430.656246  1696.955715  396.476560   
                image_1263.jpg  430.298607  1699.630873  396.227428   

scorer                                                                \
bodyparts                           Withers               LowestBack   
coords                                    x           y            x   
horse 2_1_2_del image_229.jpg    398.382821  482.592903   330.857988   
                image_230.jpg    402.516995  481.214845   343.260508   
                image_231.jpg    425.254949  479.147758   353.595942   
                image_232.jpg    427.322036  479.836787   358.419144   
                image_233.jpg    436.279412  475.702614   379.779041   
...                                     ...         ...          ...   
                image_1259.jpg  1536.776353  472.355030  1496.615234   
                image_1260.jpg  1541.761719  473.500000  1503.024747   
                image_1261.jpg  1549.564467  473.379336  1505.003899   
                image_1262.jpg  1550.049805  473.039062  1509.678711   
                image_1263.jpg  1553.966810  472.546875  1514.240228   

scorer                                      ...                             \
bodyparts                                   ... Rear_Fetlock_R               
coords                                   y  ...              x           y   
horse 2_1_2_del image_229.jpg   500.507655  ...     151.710470  862.936863   
                image_230.jpg   501.196684  ...     149.643384  859.491718   
                image_231.jpg   501.196684  ...     150.332413  855.357545   
                image_232.jpg   499.818626  ...     157.911731  860.869776   
                image_233.jpg   501.885712  ...     150.332413  855.357545   
...                                    ...  ...            ...         ...   
                image_1259.jpg  489.047739  ...    1458.749023  842.376753   
                image_1260.jpg  490.166667  ...    1457.451816  844.333333   
                image_1261.jpg  489.967878  ...    1459.300781  845.239166   
                image_1262.jpg  489.445312  ...    

In [372]:
hdf_sum[:300].to_hdf('/RnD/horse/2_1_2_del/CollectedData_teamDLC_sum.h5', key='df_with_missing', mode='w')

In [382]:
h_bbox10 = pd.read_csv("/RnD/horse/horse_csv/3_2_horse_bbox.csv")

In [384]:
h_bbox10.sort_index('filename', key=lambda x: x.map(extract_number))

TypeError: DataFrame.sort_index() takes 1 positional argument but 2 positional arguments (and 1 keyword-only argument) were given

In [386]:
h_bbox10.drop_duplicates()

,filename,x,y,w,h
0,image_434.jpg,24,366,1002,651
1,image_420.jpg,1,406,612,587
2,image_470.jpg,1053,334,866,657
3,image_441.jpg,213,392,1007,618
4,image_7.jpg,724,403,910,530
...,...,...,...,...,...
135,image_2655.jpg,1783,473,113,150
136,image_54.jpg,0,462,218,455
137,image_2685.jpg,1833,489,86,140
138,image_411.jpg,1,381,339,610
